In [1]:
import matplotlib
import numpy as np
from matplotlib import pyplot as plt
import copy
import random
import math
%matplotlib inline

In [2]:
class SmartMarket(object):
    
    def __init__(self):
        self.items = []
        self.bids = []
        self.name_to_id = {}
        self.normalisation = 1
    
    def AddItem(self, name, min_price):
        self.name_to_id[name] = len(self.items)
        self.items.append(min_price)
        
        return self.name_to_id[name]
        
    def AddSingleBid(self, items, price):
        all_items = []
        for id in items:
            if isinstance(id, str):
                id = self.name_to_id[id]
            all_items.append(id)
            
        n = len(self.bids)
        self.bids.append({"items": all_items, "price": price, "excludes":[]})
        return n
    
    def AddBids(self, bids):
        excludes = []
        for b,p in bids:
            n = self.AddSingleBid(b,p)
            excludes.append(n)
        
        for i in excludes:
            self.bids[i]["excludes"] = copy.copy(excludes)

    def BuildGraph(self):
        # Building graph|
        self.local_fields = [ 0 for i in range(len(self.bids))]
        self.couplings = [ [0]*len(self.bids) for i in range(len(self.bids))]
        self.active = [random.randint(0,1) for i in range(len(self.bids)) ]
        
        for i in range(len(self.bids)):
            bid1 = self.bids[i]
            
            e = bid1["price"]
            for n in range(len(self.items)):
                if n in bid1["items"]:
                    e -= self.items[n]
                
            self.local_fields[i] = e
            
            for j in range(i+1,len(self.bids)):
                bid2 = self.bids[j]
                
                coupling = 0
                if i in bid2["excludes"]:
                    coupling = 10
                    
                for n in range(len(self.items)):
                    if n in bid1["items"] and n in bid2["items"]:
                        coupling += (bid1["price"] + bid2["price"])
                
                self.couplings[i][j] = self.couplings[j][i] = -coupling

    def TotalBenefit(self):
        reward = 0
        for i in range(len(self.bids)):
            a1 = self.active[i]
            reward += a1*self.local_fields[i]
            
            for j in range(len(self.bids)):
                a2 = self.active[j]
                reward += a1*a2*self.couplings[i][j]
                
        return reward
    
    def SelectBid(self, bid):        
        for j in range(len(self.bids)):
            if self.couplings[bid][j] != 0:
                self.active[j] = 0
                
        self.active[bid] = 1
    
    def Mine(self, hash_value, runtime):
        random.seed(hash_value)
        self.BuildGraph()
        beta_start = 0.01
        beta_end = 1
        db = (beta_end-beta_start)/runtime
        beta = beta_start
        
        rejected = 0
        for s in range(runtime):
            for i in range(len(self.bids)):

                oldactive = copy.copy(self.active)
                oldreward = self.TotalBenefit()

                nn = random.randint(1, 3)
                for gg in range(nn):
                    n = random.randint(0, len(self.bids)-1)

                    if self.active[n] == 1:
                        self.active[n] = 0
                    else:
                        self.active[n] = 1

                #self.SelectBid( n )
                newreward = self.TotalBenefit()
                de = oldreward - newreward

                if random.random() < math.exp(-beta*de):
                    pass
                    #print(oldreward, " => ", newreward, ":",de,beta,  math.exp(-beta*de))
                else:
                    self.active = oldactive
                    rejected += 1

            beta += db
        #print(rejected/ runtime/len(self.bids))

In [3]:
market = SmartMarket()

market.AddItem("car", 0)
market.AddItem("hotel", 0)
market.AddItem("cinema", 0)
market.AddItem("opera",0)

market.AddSingleBid(["car"], 10)
market.AddSingleBid(["hotel"],8)
market.AddSingleBid(["cinema"],7)
market.AddSingleBid([ "hotel", "car","cinema"], 29)
market.AddBids([ ([ "hotel", "car","cinema"], 20), ([ "hotel", "car","opera"], 30) ])
market.AddSingleBid(["opera"],7)

market.BuildGraph()
print(market.TotalBenefit())
market.SelectBid( 0 )
market.SelectBid( 1 )
market.SelectBid( 2 )
#market.SelectBid( 3 )
print(market.TotalBenefit())
print("---")
for i in range(10):
    market.Mine(23*i,100)
    print(market.TotalBenefit())
    
print(np.array(market.local_fields))
print(np.array(market.couplings))

-790
25
---
37
37
37
32
36
37
37
32
36
37
[10  8  7 29 20 30  7]
[[   0    0    0  -39  -30  -40    0]
 [   0    0    0  -37  -28  -38    0]
 [   0    0    0  -36  -27    0    0]
 [ -39  -37  -36    0 -147 -118    0]
 [ -30  -28  -27 -147    0 -110    0]
 [ -40  -38    0 -118 -110    0  -37]
 [   0    0    0    0    0  -37    0]]


In [4]:
import fetch
import fetch.auctions
import fetch.byte_array

In [6]:
items = []

item_id = 0
seller_id = 1
min_price = 0
items.append(fetch.auctions.Item(item_id, seller_id, min_price))
car = items[-1]

items.append(fetch.auctions.Item(1,1,0))
hotel = items[-1]

items.append(fetch.auctions.Item(2,1,0))
cinema = items[-1]

items.append(fetch.auctions.Item(3,1,0))
opera = items[-1]

bids = []
bids.append(fetch.auctions.Bid(0, [car], 10, 1))
bids.append(fetch.auctions.Bid(1, [hotel], 8, 1))
bids.append(fetch.auctions.Bid(2, [cinema], 7, 1))
bids.append(fetch.auctions.Bid(3, [car, hotel, cinema], 29, 1))
# [ ([ "hotel", "car","cinema"], 20), ([ "hotel", "car","opera"], 30) ]
b1 = fetch.auctions.Bid(4, [car, hotel, cinema], 20, 1)
b2 = fetch.auctions.Bid(5, [car, hotel, opera], 30, 1)
b1.Excludes().append(b2)
b2.Excludes().append(b1)

bids.append(b1)
bids.append(b2)
bids.append(fetch.auctions.Bid(6, [opera], 7, 1))

ba1 = fetch.byte_array.ByteArray("StartBlockId")
ba2 = fetch.byte_array.ByteArray("ExecuteBlockId")
ca  = fetch.auctions.CombinatorialAuction(ba1, ba2)

for cur_item in items:
    assert(ca.AddItem(cur_item) == 0)
    
for cur_bid in bids:
    assert(ca.PlaceBid(cur_bid) == 0)
print("----")
#for i in range(10):
ca.BuildGraph()
ca.SelectBid(2)
ca.SelectBid(5)
#ca.SelectBid(1)
#ca.SelectBid(2)
#ca.SelectBid(3)
#ca.SelectBid(4)
print(ca.TotalBenefit())
print("---")
ca.Mine(23,100)
print(ca.TotalBenefit())
print("xxx")
for i in range(10):
    ca.Mine(23*i,100)
    print(ca.TotalBenefit())
print("---")
print(ca.local_fields().ToNumpy())
print(ca.couplings().ToNumpy())

----
37.0
---
-347.0
xxx
-404.0
-347.0
-926.0
-837.0
-388.0
14.0
-95.0
-390.0
-30.0
-20.0
---
[10.  8.  7. 29. 20. 30.  7.]
[[   0.   -0.   -0.  -39.  -30.  -40.   -0.]
 [  -0.    0.   -0.  -37.  -28.  -38.   -0.]
 [  -0.   -0.    0.  -36.  -27.   -0.   -0.]
 [ -39.  -37.  -36.    0. -147. -118.   -0.]
 [ -30.  -28.  -27. -147.    0. -100.   -0.]
 [ -40.  -38.   -0. -118. -100.    0.  -37.]
 [  -0.   -0.   -0.   -0.   -0.  -37.    0.]]


In [6]:
dir(fetch)

['__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'auctions',
 'byte_array',
 'image',
 'math',
 'memory',
 'ml',
 'random',
 'vectorize']